In [64]:
!pip install --upgrade weaviate-client

     ---------------------------------------- 0.0/81.8 kB ? eta -:--:--
     ---------------------------------------- 81.8/81.8 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/292.9 kB ? eta -:--:--
   ---------------------------------------- 292.9/292.9 kB 8.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.9 kB ? eta -:--:--
   ---------------------------------------- 75.9/75.9 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.9 kB ? eta -:--:--
   ---------------------------------------- 76.9/76.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------------------------------- ------ 0.9/1.1 MB 28.7 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 22.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/394.2 kB ? eta -:--:--
   --------------------------------------- 394.2/394.2 kB 24.0 MB/s eta 0:00:00
   -------------------------------

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.0.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.6.0 which is incompatible.
embedchain 0.1.31 requires langchain<0.0.337,>=0.0.336, but you have langchain 0.1.4 which is incompatible.
embedchain 0.1.31 requires tiktoken<0.5.0,>=0.4.0, but you have tiktoken 0.5.2 which is incompatible.
jupyter-ai 2.0.1 requires importlib-metadata~=5.2.0, but you have impor

# Import the Schema Classes for Article & Author

In [55]:
import weaviate
client = weaviate.Client("http://localhost:8080")

schema = {
  "classes": [
    {
      "class": "Article",
      "description": "A class to store articles",
      "properties": [
        {"name": "title", "dataType": ["string"], "description": "The title of the article"},
        {"name": "content", "dataType": ["text"], "description": "The content of the article"},
        {"name": "datePublished", "dataType": ["date"], "description": "The date the article was published"},
        {"name": "url", "dataType": ["string"], "description": "The URL of the article"}
      ]
    },
    {
      "class": "Author",
      "description": "A class to store authors",
      "properties": [
        {"name": "name", "dataType": ["string"], "description": "The name of the author"},
        {"name": "articles", "dataType": ["Article"], "description": "The articles written by the author"}
      ]
    }
  ]
}
client.schema.delete_class('Article')
client.schema.delete_class('Author')
client.schema.create(schema)

# JSON data to be Ingested

data = [
    {
        "class": "Article",
        "properties": {
            "title": "LangChain: OpenAI + S3 Loader",
            "content": "This article discusses the integration of LangChain with OpenAI and S3 Loader...",
            "url": "https://blog.min.io/langchain-openai-s3-loader/"
        }
    },
    {
        "class": "Article",
        "properties": {
            "title": "MinIO Webhook Event Notifications",
            "content": "Exploring the webhook event notification system in MinIO...",
            "url": "https://blog.min.io/minio-webhook-event-notifications/"
        }
    },
    {
        "class": "Article",
        "properties": {
            "title": "MinIO Postgres Event Notifications",
            "content": "An in-depth look at Postgres event notifications in MinIO...",
            "url": "https://blog.min.io/minio-postgres-event-notifications/"
        }
    },
    {
        "class": "Article",
        "properties": {
            "title": "From Docker to Localhost",
            "content": "A guide on transitioning from Docker to localhost environments...",
            "url": "https://blog.min.io/from-docker-to-localhost/"
        }
    }
]

for item in data:
    client.data_object.create(
        data_object=item["properties"],  # Ensure this is a dictionary of properties
        class_name=item["class"]  # The class name as a string
    )

In [61]:
import weaviate

client = weaviate.Client("http://192.168.0.25:8080")
result = client.backup.create(
  backup_id="my-very-first-backup",
  backend="s3",
  # include_classes=["Article", "Author"],  # specify classes to include or omit this for all classes
  wait_for_completion=True,
)
print(result)

{'backend': 's3', 'classes': ['Article', 'Author'], 'id': 'my-very-first-backup', 'path': 's3://weaviate-backups/my-very-first-backup', 'status': 'SUCCESS'}


In [76]:
result = client.backup.restore(
  backup_id="my-very-first-backup",
  backend="s3",
  exclude_classes="Article",
  wait_for_completion=True,
)

print(result)

BackupFailedException: Backup restore failed: {'backend': 's3', 'classes': ['Author'], 'id': 'my-very-first-backup', 'path': 's3://weaviate-backups/my-very-first-backup', 'status': 'FAILED', 'error': 'restore class Author: already exists'}